In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('yelp').getOrCreate()

In [5]:
# Import csv of spam and ham (not spam)
start_data = spark.read.format("csv").\
    option("header", "true").\
    option("delimiter", "\t").\
    load("yelp_reviews.tsv")
start_data.show()

+--------+--------------------+
|   class|                text|
+--------+--------------------+
|positive|Wow... Loved this...|
|negative|  Crust is not good.|
|negative|Not tasty and the...|
|positive|Stopped by during...|
|positive|The selection on ...|
|negative|Now I am getting ...|
|negative|Honeslty it didn'...|
|negative|The potatoes were...|
|positive|The fries were gr...|
|positive|      A great touch.|
|positive|Service was very ...|
|negative|  Would not go back.|
|negative|The cashier had n...|
|positive|I tried the Cape ...|
|negative|I was disgusted b...|
|negative|I was shocked bec...|
|positive| Highly recommended.|
|negative|Waitress was a li...|
|negative|This place is not...|
|negative|did not like at all.|
+--------+--------------------+
only showing top 20 rows



In [7]:
# Create a length column to be used as a future feature 
from pyspark.sql.functions import length
data = start_data.withColumn('length', length(start_data['text']))
data.show()

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|positive|Wow... Loved this...|    24|
|negative|  Crust is not good.|    18|
|negative|Not tasty and the...|    41|
|positive|Stopped by during...|    87|
|positive|The selection on ...|    59|
|negative|Now I am getting ...|    46|
|negative|Honeslty it didn'...|    37|
|negative|The potatoes were...|   111|
|positive|The fries were gr...|    25|
|positive|      A great touch.|    14|
|positive|Service was very ...|    24|
|negative|  Would not go back.|    18|
|negative|The cashier had n...|    99|
|positive|I tried the Cape ...|    59|
|negative|I was disgusted b...|    62|
|negative|I was shocked bec...|    50|
|positive| Highly recommended.|    19|
|negative|Waitress was a li...|    38|
|negative|This place is not...|    51|
|negative|did not like at all.|    20|
+--------+--------------------+------+
only showing top 20 rows



### Feature Transformations


In [18]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

# Create the data processing pipeline functions here
sentiment_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [20]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[sentiment_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [21]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data)
cleaned = cleaner.transform(data)

In [22]:
# Show label of and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[33933,69...|
|  1.0|(262145,[150903,1...|
|  1.0|(262145,[63367,11...|
|  0.0|(262145,[6286,272...|
|  0.0|(262145,[6979,911...|
|  1.0|(262145,[24661,34...|
|  1.0|(262145,[101702,2...|
|  1.0|(262145,[3645,855...|
|  0.0|(262145,[53777,13...|
|  0.0|(262145,[138356,2...|
|  0.0|(262145,[24113,20...|
|  1.0|(262145,[172477,1...|
|  1.0|(262145,[36200,40...|
|  0.0|(262145,[18098,83...|
|  1.0|(262145,[89493,95...|
|  1.0|(262145,[86431,10...|
|  0.0|(262145,[31704,21...|
|  1.0|(262145,[27707,65...|
|  1.0|(262145,[12329,61...|
|  1.0|(262145,[8287,208...|
+-----+--------------------+
only showing top 20 rows



In [23]:
# Break data down into a training set and a testing set
(training, testing) = cleaned.randomSplit([0.7, 0.3])

In [24]:
from pyspark.ml.classification import NaiveBayes

# Create a Naive Bayes model and fit training data
nb = NaiveBayes(smoothing=1.0, modelType='multinomial')
spam_predictor = nb.fit(training)

In [25]:
# Tranform the model with the testing data
test_results = spam_predictor.transform(testing)
test_results.show(5)

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|negative|!....THE OWNERS R...|   120|  1.0|[!....the, owners...|[!....the, owners...|(262144,[14,12946...|(262144,[14,12946...|(262145,[14,12946...|[-1279.4307331242...|[0.99999999999589...|       0.0|
|negative|"As for the ""mai...|    40|  1.0|["as, for, the, "...|["as, ""mains,"",...|(262144,[10003,11...|(262144,[10003,11...|(262145,[10003,11...|[-374.62077074961...|[0.93545079726937.

In [26]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print(f"Accuracy of model at predicting reviews was: {acc}")

Accuracy of model at predicting reviews was: 0.693832571514984


In [27]:
spark.stop()